In [59]:
import pandas as pd
df = pd.read_excel('./학생이름_꿈_취미_가공_240117.xlsx')
df.head()

,Unnamed: 0,mgmt_id,class_mgmt_id,grade,class,name,년,월,음력,설명 1,설명 2,링크,전체 링크,QR용 str,꿈,search_keywords,일생의 키워드,preprocessed_for_bow,키워드
0,0,1,1,1반,1,고해인,2021,5,NaN,엄마가 힘들다고 하면 해바라기 같은 웃음을 띠고 꼭 안아주는,세계여행을 가고 싶어하던 해인이는 간호사라는 꿈을 가지고 있었습니다.,http://www.416memory.org/collection_col1/view/...,http://www.416memory.org/collection_col1/view/...,001. 1반 1번 고해인,간호사,"간호사, 여행가, 베스트프랜드, 미소천사",NaN,엄마가 힘들다고 하면 해바라기 같은 웃음을 띠고 꼭 안아주는 세계여행을 가고 싶어하...,"엄마가, 같은, 띠고, 안아주는, 세계여행을, 가고, 싶어하던, 간호사라는, 꿈을,..."
1,1,2,2,1반,5,김민지,2021,9,NaN,"아빠의 보배, 아빠의 가장 친한 친구이자 끝없는 잔소리꾼인 '꽁민지'","단원고 2학년 1반, 아빠의 보배였던 민지는 가수를 꿈꾸고 있었습니다.",http://www.416memory.org/collection_col1/view/...,http://www.416memory.org/collection_col1/view/...,002. 1반 5번 김민지,가수,"가수, 잔소리꾼, 아빠의 보물",NaN,아빠의 보배 아빠의 가장 친한 친구이자 끝없는 잔소리꾼인 꽁민지 단원고 2학년 1반...,"아빠의, 가장, 끝없는, 꽁민지, 단원고, 2학년, 1반, 민지는, 가수를, 꿈꾸고"
2,2,3,3,1반,6,김민희,2021,4,NaN,초등학교 때 점토 교육강사 자격증을 딸 정도로 손재주가 좋고 만들기를 좋아하는,민희는 사서가 되고 싶은 꿈을 가지고 있었습니다.,http://www.416memory.org/collection_col1/view/...,http://www.416memory.org/collection_col1/view/...,003. 1반 6번 김민희,사서,"사서, 교육강사,클레이, 점토, 자격증",NaN,초등학교 때 점토 교육강사 자격증을 딸 정도로 손재주가 좋고 만들기를 좋아하는 민희...,"교육강사, 자격증을, 손재주가, 만들기를, 민희는, 사서가, 되고, 싶은, 꿈을, 가지고"
3,3,4,4,1반,7,김수경,2021,4,NaN,"친구들의 고민해결사, '오케이 걸'",수경이는 분위기 메이커로 모두에게 즐거움을 주고 고민을 들어주던 좋은 친구였습니다.,http://www.416memory.org/collection_col1/view/...,http://www.416memory.org/collection_col1/view/...,004. 1반 7번 김수경,국가정보요원,"국가정보요원, 고민상담사, 분위기 메이커",NaN,친구들의 고민해결사 오케이 걸 수경이는 분위기 메이커로 모두에게 즐거움을 주고 고민...,"고민해결사, 오케이, 수경이는, 분위기, 메이커로, 모두에게, 주고, 고민을, 들어..."
4,4,5,5,1반,8,김수진,2021,2,NaN,"친구들에게 인기가 좋아 별명도 많은 ('학다리', '초코', '디오', '수잔', ...",인기 많은 수진이는 요리사의 꿈을 키우고 있었습니다.,http://www.416memory.org/collection_col1/view/...,http://www.416memory.org/collection_col1/view/...,005. 1반 8번 김수진,요리사,"요리사, 인기녀, 별명 부자",NaN,친구들에게 인기가 좋아 별명도 많은 학다리 초코 디오 수잔 코난 인기 많은 수진이는...,"인기가, 좋아, 별명도, 많은, 디오, 수잔, 인기, 수진이는, 요리사의, 꿈을"


In [8]:
%pip install sqlalchemy sql psycopg2-binary


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
import psycopg2
import pandas as pd
import numpy as np
import csv

import sql
from sqlalchemy import create_engine
user = 'root' # postgresql DB user ID
password = '1efqtf2dlrableek' # password
host_product = 'svc.sel5.cloudtype.app' # 서버 IP 
dbname = 'postgres' # DB Name
port='30056' # port number

product_connection_string = "dbname={dbname} user={user} host={host} password={password} port={port}".format(dbname=dbname,
                                    user=user,
                                    host=host_product,
                                    password=password,
                                    port=port) 


In [61]:
try:
    error_key = ''
    conn = psycopg2.connect(product_connection_string)
    with conn.cursor() as curs:
      for person in df.itertuples():
        if type(person.search_keywords) != float:
          mgmt_id = f"{person.mgmt_id}"
          find_student_query = f"SELECT id,mgmt_id FROM students s where s.mgmt_id = '{mgmt_id}'"
          curs.execute(find_student_query)
          student_from_db = curs.fetchone()
          student_id = student_from_db[0]
          keywords = list(map(lambda x : x.strip() , person.search_keywords.split(',')))
          for keyword in keywords:  
            error_key = keyword
            find_keyword_query = f"SELECT id FROM keyword where name = '{keyword}'"
            curs.execute(find_keyword_query)
            find_key = curs.fetchone()
            keyword_id = find_key[0]
            sql = f"INSERT INTO student_keyword(student_id,keyword_id) VALUES ('{student_id}','{keyword_id}')"
            curs.execute(sql)
      conn.commit()
except Exception as e:
    print("I am unable to connect to the database",e,error_key)
    conn.rollback()

In [64]:
try:
    error_key = ''
    conn = psycopg2.connect(product_connection_string)
    with conn.cursor() as curs:
      for person in df.itertuples():
        if type(person.search_keywords) != float:
          mgmt_id = f"{person.mgmt_id}"
          find_student_query = f"SELECT id,mgmt_id FROM students s where s.mgmt_id = '{mgmt_id}'"
          curs.execute(find_student_query)
          student_from_db = curs.fetchone()
          student_id = student_from_db[0]
          keywords = list(map(lambda x : x.strip() , person.search_keywords.split(',')))
          keyword = keywords[0]
          error_key = keyword
          find_keyword_query = f"SELECT id FROM keyword where name = '{keyword}'"
          curs.execute(find_keyword_query)
          find_key = curs.fetchone()
          keyword_id = find_key[0]
          sql = f"update students set title_keyword_id = '{keyword_id}' where id = '{student_id}'"
          curs.execute(sql)
      conn.commit()
except Exception as e:
    print("I am unable to connect to the database",e,error_key)
    conn.rollback()

간호사
가수
사서
국가정보요원
요리사
간호사
가수
화가
일본어 교사
스튜어디스
판사
일러스트레이터
의사
국어선생님
네일 아티스트
유치원 선생님
엄마 껌딱지
카메라 감독
웨딩드레스 디자이너
군인
연예인
의상 디자이너
화가
비디오 저널리스트
광고 디자이너
영어 선생님
역사학자
방송 작가
방송국 PD
사업가
국어선생님
소설가
음악 심리치료사
제과제빵사
유치원 선생님
경찰관
메이크업 아티스트
국제 구호활동가
박물관 큐레이터
수화통역사
한문 선생님
유치원 선생님
온라인 패션몰 운영
변호사
선생님
연극배우
방송국 PD
국제 구호활동가
음악 선생님
얄루
디자이너
음악가
유치원 선생님
구두 디자이너
유학생
만화가
패션 디자이너
경찰관
세무 공무원
뮤지컬 배우
방송 작가
여군
장교
역사 선생님
뮤지컬 배우
유치원 선생님
사색가
사회복지사
중국어 번역가
작곡가
경영인
축구
체육 선생님
아빠 바라기
칼리 아르니스 무술 사범
웹 디자이너
장교
마술사
법관
방송작가
역사 선생님
경찰관
ADHD 밴드 리더
축구 선수
예술가
모델
요리사
중국어 능력자
태권도 사범
목사
패셔니스타
검도 사범
자동차 디자이너
만화가
소설가
합기도
애니메이션 감독
체육 선생님
심리학자
작곡가
교과 도우미
특수부대 군인
역사 선생님
범죄심리분석관
사육사
합기도 관장
프로게이머
경호원
사제
바리스타
대건 안드레아
이종격투기 선수
가수
세무 공무원
관광 가이드
체육 선생님
국밥집 사장님
헤어디자이너
실용미술
사회복지사
역사 선생님
제과제빵사
검사
경찰관
ROTC
모델
공무원
연극배우
사업가
체육 선생님
건축 설계사
헤어아티스트
영어 선생님
육상
축구
외교관
축구 해설가
국어 선생님
그래픽 디자이너
뮤지션
회계사
우주 과학자
패션 디자이너
호텔 요리사
아나운서
장교
경호원
법관
체육 선생님
다국적 기업 취직
과학자
간지쿵
누나들의 보디가드
수학 선생님
기타 독학
전기 엔지니어
수학 선생님
수학선생님
태권도
셰프
경호원
장교
요리사
건축가
형제 요리사
프로게이머
경찰관
야구 선수
긍정맨
간호사
특전사
과학선생님
곤충

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import csv
import pandas as pd
import sql
from sqlalchemy import create_engine
user = 'root' # postgresql DB user ID
password = '1efqtf2dlrableek' # password
host_product = 'svc.sel5.cloudtype.app' # 서버 IP 
dbname = 'postgres' # DB Name
port='30056' # port number

product_connection_string = "dbname={dbname} user={user} host={host} password={password} port={port}".format(dbname=dbname,
                                    user=user,
                                    host=host_product,
                                    password=password,
                                    port=port) 


In [20]:
df = pd.read_excel('./학생이름_꿈_취미_가공_240117.xlsx')
df['birthday'].head()

0    2024-05-14 00:00:00
1    2024-09-11 00:00:00
2    2024-04-19 00:00:00
3    2024-04-22 00:00:00
4    2024-02-08 00:00:00
Name: birthday, dtype: object

In [31]:
from datetime import datetime

conn = psycopg2.connect(product_connection_string)
try:
  with conn.cursor() as curs:
    for row in df.itertuples():
      if type(row.birthday) == datetime:
        month, day = row.birthday.date().strftime('%m %d').split(' ')
        is_lunar = type(row.is_lunar) == str
        where_id = row.mgmt_id 
        birthday = f'1997-{month}-{day}'
        q = f"UPDATE students SET birthday = '{birthday}', is_lunar_birth = {is_lunar} where mgmt_id = {where_id}"
        curs.execute(q)
    conn.commit()
except Exception as e:
  print(e)
  conn.rollback()